In [1]:
!aws s3 ls

2019-08-05 22:10:00 aws-deepracer-25b1a0a1-1b22-4c0a-a96d-473263ffa6b6
2020-05-09 13:53:16 msc-project-pickle
2020-05-10 11:25:45 sagemaker-studio-937669247751-yos4c5c3k0q


In [2]:
!aws s3 cp s3://msc-project-pickle/X.pickle X.pickle

download: s3://msc-project-pickle/X.pickle to ./X.pickle           


In [3]:
!aws s3 cp s3://msc-project-pickle/y.pickle y.pickle

download: s3://msc-project-pickle/y.pickle to ./y.pickle         


In [1]:
import pickle
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.utils import parallel_backend

In [2]:
with open("X.pickle", "rb") as fx:
    X = pickle.load(fx)

In [3]:
with open("y.pickle", "rb") as fy:
    y = pickle.load(fy)

In [5]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC(gamma="scale")
clf = GridSearchCV(svc, parameters, cv=3, verbose=2, n_jobs=2)
with parallel_backend('multiprocessing'):
    clf.fit(X, y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=2)]: Using backend MultiprocessingBackend with 2 concurrent workers.


[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=11.8min
[CV] C=1, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=11.9min
[CV] C=1, kernel=rbf .................................................
[CV] ............................... C=1, kernel=linear, total=12.3min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total=15.5min
[CV] C=1, kernel=rbf .................................................
[CV] .................................. C=1, kernel=rbf, total=15.2min
[CV] C=10, kernel=linear .............................................
[CV] .................................. C=1, kernel=rbf, total=15.8min
[CV] C=10, kernel=linear .............................................
[CV] .

[Parallel(n_jobs=2)]: Done  12 out of  12 | elapsed: 83.8min finished


In [8]:
clf.best_params_

{'C': 10, 'kernel': 'rbf'}

In [6]:
clf.best_score_

0.971818585473312

In [12]:
[str(i) for i in clf.best_params_.values()]

['10', 'rbf']

In [13]:
model_params = "_".join([str(i) for i in clf.best_params_.values()])

In [14]:
with open(f"svm_{model_params}.pickle", "wb") as model:
    pickle.dump(clf, model)

In [15]:
!aws s3 cp svm_10_rbf.pickle s3://msc-project-pickle/svm_10_rbf.pickle

upload: ./svm_10_rbf.pickle to s3://msc-project-pickle/svm_10_rbf.pickle
